In [156]:
import basedosdados as bd
#Lendo a base para as questões 1 a 5
#Optei por realizar uma consulta "minimamente viável" em que fosse possível responder todas a maior quantidade de perguntas possíveis sem ler todos os dados
query = """
SELECT
  tb_chamado.*,
  tb_bairro.nome,
  tb_bairro.subprefeitura
FROM datario.adm_central_atendimento_1746.chamado tb_chamado
LEFT JOIN datario.dados_mestres.bairro tb_bairro
  ON tb_chamado.id_bairro = tb_bairro.id_bairro
WHERE DATE(tb_chamado.data_inicio) = '2023-04-01'
"""

df = bd.read_sql(query=query, billing_project_id="teste-433114")
respostas = {}

Downloading: 100%|██████████| 1756/1756 [00:00<00:00, 2138.66rows/s]


# Perguntas 1 a 5
1. Quantos chamados foram abertos no dia 01/04/2023?
2. Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?
3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?
4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?
5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

In [157]:
#Perguntas 1 a 5
#Pergunta 01
qtd_chamados = df['id_chamado'].count()
respostas.update({"pergunta_01": qtd_chamados})

#Pergunta 02
tipo_chamado = df.groupby('tipo')['id_chamado'].count().sort_values(ascending=False).index[0]
respostas.update({"pergunta_02": tipo_chamado})

#Pergunta 03
bairro_chamado = df.groupby('nome')['id_chamado'].count().sort_values(ascending=False).index[0:3]
respostas.update({"pergunta_03": bairro_chamado})

#Pergunta 04
subprefeitura_chamado = df.groupby(['subprefeitura'])['id_chamado'].count().sort_values(ascending=False).index[0]
respostas.update({"pergunta_04": subprefeitura_chamado})

#Pergunta 05
chamados_null = df[df['id_bairro'].isnull() | df['subprefeitura'].isnull()]['id_chamado'].count()
respostas.update({"pergunta_05": chamados_null})
#Aparentemente, essas linhas não possuem bairro por serem chamados ligados a transportes ou problemas online, não possuindo logradouro.

# Perguntas 6 a 10
6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?
7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).
8. Quantos chamados desse subtipo foram abertos em cada evento?
9. Qual evento teve a maior média diária de chamados abertos desse subtipo?
10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

In [158]:
#Lendo a base para as questões 6 a 10
query = """
SELECT
  tb_chamado.id_chamado,
  DATE(tb_chamado.data_inicio) AS data_inicio,
  tb_chamado.subtipo
FROM datario.adm_central_atendimento_1746.chamado tb_chamado
WHERE DATE(tb_chamado.data_inicio) BETWEEN '2022-01-01' AND '2023-12-31'
  AND tb_chamado.subtipo = 'Perturbação do sossego'
"""
#Realizei a consulta da tabela de eventos separada e mesclei com a principal para fazer diferente da tarefa de sql
df = bd.read_sql(query=query, billing_project_id="teste-433114")
df_eventos = bd.read_sql(query="SELECT * FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`", billing_project_id="teste-433114")
df_eventos['dias_evento'] = (df_eventos['data_final'] - df_eventos['data_inicial']).dt.days + 1 

Downloading: 100%|██████████| 8/8 [00:00<00:00, 49.96rows/s]


In [159]:
#Perguntas 6 a 10
#Pergunta 06
qtd_chamados = df['id_chamado'].count()
respostas.update({"pergunta_06": qtd_chamados})

#Pergunta 07
import pandas as pd #chamando a biblioteca pandas porque terei que apelar
chamados_eventos = []
for _, row in df_eventos.iterrows(): #Iterando linha a linha do dataframe de eventos
  inicio = row['data_inicial'] #data inicial do evento
  fim = row['data_final'] #data final do evento
  evento = row['evento'] #nome do evento
  #criando tabela temporária para realizar o loop
  temp = df[(df['data_inicio'] >= inicio) & (df['data_inicio'] <= fim)].copy() #Filtrando eventos que ocorreram entre as datas do evento
  temp['evento'] = evento
  chamados_eventos.append(temp) #juntando à lista de chamados que aconteceram na data do evento

chamados_eventos = pd.concat(chamados_eventos) #Concatenando todas as semi-tableas geradas no loop
chamados_eventos = chamados_eventos.drop_duplicates(subset=['id_chamado']) #removendo id's duplicados
respostas.update({"pergunta_07": chamados_eventos})

#Pergunta 08
qtd_chamados_evento = respostas['pergunta_07'].groupby('evento')['id_chamado'].count().sort_values(ascending=False)
respostas.update({"pergunta_08": qtd_chamados_evento.head(1)})

#Pergunta 09
soma_dias_evento = df_eventos.drop_duplicates(subset=['evento', 'data_inicial', 'data_final']).groupby('evento')['dias_evento'].sum()
med_dias_chamados = (qtd_chamados_evento/soma_dias_evento).sort_values(ascending=False)
respostas.update({"pergunta_09": med_dias_chamados.head(1)})

#Pergunta 10
dias_periodo = (df['data_inicio'].max() - df['data_inicio'].min()).days + 1
med_dias_chamados['med_total_chamados'] = qtd_chamados/dias_periodo
respostas.update({"pergunta_10": med_dias_chamados})

# Respostas

In [160]:
#Exibindo as respostas
for pergunta, resposta in respostas.items():
  print(f"Resposta da {pergunta}: {resposta}")

Resposta da pergunta_01: 1756
Resposta da pergunta_02: Estacionamento irregular
Resposta da pergunta_03: Index(['Campo Grande', 'Tijuca', 'Barra da Tijuca'], dtype='object', name='nome')
Resposta da pergunta_04: Zona Norte
Resposta da pergunta_05: 73
Resposta da pergunta_06: 42830
Resposta da pergunta_07:       id_chamado data_inicio                 subtipo       evento
24389   17662396  2022-09-03  Perturbação do sossego  Rock in Rio
24398   17663234  2022-09-03  Perturbação do sossego  Rock in Rio
24417   17663056  2022-09-03  Perturbação do sossego  Rock in Rio
24423   17661947  2022-09-03  Perturbação do sossego  Rock in Rio
24461   17659216  2022-09-02  Perturbação do sossego  Rock in Rio
...          ...         ...                     ...          ...
32905   18078426  2022-12-31  Perturbação do sossego    Reveillon
32925   18079582  2022-12-31  Perturbação do sossego    Reveillon
32930   18078672  2022-12-31  Perturbação do sossego    Reveillon
32931   18075943  2022-12-30  Per